In [3]:
import pandas as pd
import math

class Bayesclassifier():
    def __init__(self):
        pass
    def train(self, X, y):
        N = len(y)
        self.classes = self.count_list(y)
        self.class_num = len(self.classes)
        self.classes_p = {}
        for c, n in self.classes.items():
            self.classes_p[c] = n / N

        self.discrete_good_p = []
        self.discrete_bad_p = []
        for i in range(6):
            attr_with_good = []
            attr_with_bad = []
            for j in range(N):
                if y[j] == "是":
                     attr_with_good.append(X[j][i])
                else:
                    attr_with_bad.append(X[j][i])
            unique_with_good = self.count_list(attr_with_good)
            unique_with_bad = self.count_list(attr_with_bad)
            self.discrete_good_p.append(self.discrete_p(unique_with_good, self.classes["是"]))
            self.discrete_bad_p.append(self.discrete_p(unique_with_bad, self.classes["否"]))
        self.good_mus = []
        self.good_vars = []
        self.bad_mus = []
        self.bad_vars = []
        for i in range(2):
            attr_with_good = []
            attr_with_bad = []
            for j in range(N):
                if y[j] == "是":
                    attr_with_good.append(X[j][i+6])
                else:
                    attr_with_bad.append(X[j][i+6])
            good_mu, good_var = self.mu_var(attr_with_good)
            bad_mu, bad_var = self.mu_var(attr_with_bad)
            self.good_mus.append(good_mu)
            self.good_vars.append(good_var)
            self.bad_mus.append(bad_mu)
            self.bad_vars.append(bad_var)

    def predict(self, x):
        p_good = self.classes_p["是"]
        p_bad = self.classes_p["否"]
        for i in range(6):
            p_good  *= self.discrete_good_p[i][x[i]]
            p_bad *= self.discrete_bad_p[i][x[i]]
        for i in range(2):
            p_good *= self.continuous_p(x[i+6], self.good_mus[i], self.good_vars[i])
            p_bad *= self.continuous_p(x[i+6], self.bad_mus[i], self.bad_vars[i])
        if p_good >= p_bad:
            return p_good, p_bad, "是"
        else:
            return p_good, p_bad, "否"
    '''归类'''
    def count_list(self, l):
        unique_dict = {}
        for e in l:
            if e in unique_dict:
                unique_dict[e] += 1
            else:
                unique_dict[e] = 1
        return unique_dict
    '''算离散的概率'''
    def discrete_p(self, d, N_class):
        new_d = {}
        for a, n in d.items():
            new_d[a] = n / N_class
        return new_d
    '''算连续的概率'''
    def continuous_p(self, x, mu, var):
        p = 1.0 / (math.sqrt(2*math.pi) * math.sqrt(var)) * math.exp(- (x-mu)**2 /(2*var))
        return p
    '''计算均值方差'''
    def mu_var(self, l):
        mu = sum(l) / float(len(l))
        var = 0
        for i in range(len(l)):
            var += (l[i]-mu)**2
        var = var / float(len(l)-1)
        return mu, var

In [8]:
if __name__=="__main__":
    bcf = Bayesclassifier()
    data = pd.read_csv("data/data24374/watermelon_3.csv", encoding="gb18030")
    X = data.values[:, 1:9]
    y = data.values[:, 9]
    '''读取表格后进行训练'''
    bcf.train(X, y)
    samp2 = ["乌黑", "蜷缩", "浊响", "清晰", "凹陷", "硬滑", 0.697, 0.460]
    yes_p, no_p, ans = bcf.predict(samp2)
    print ("预测结果为: {}, {:.7f}, {:.7f}".format(ans, yes_p, no_p))

预测结果为: 是, 0.0698383, 0.0000457
